## Project: Bank Account Manager
**Features**
1. Account Creation – name, account number, initial balance  
2. Basic Operations – deposit, withdraw, check balance  
3. Account Details  
4. Bank Info – set bank name and track total number of accounts  
5. Transfers – transfer between accounts  
6. Transaction History  
7. Authentication – PIN system for withdrawals  
8. Account Types – Savings, Checking, and Joint Accounts  

In [1]:
from abc import ABC, abstractmethod

class Account(ABC):
    
  bank_name = "CBE"
  total_accounts = 0
  accounts_registry = {} # dictionary

  @classmethod
  def get_total_accounts(cls):
    return cls.total_accounts

  @classmethod
  def get_accounts(cls):
    if not cls.accounts_registry:
      print("No accounts available")
      return False

    for acc_no, acc in cls.accounts_registry.items():
      print(acc.account_info())

  @classmethod
  def find_account(cls, account_no):
      return cls.accounts_registry.get(account_no)
    
  def __init__(self, owner, balance, account_no, PIN):
    self.owner = owner
    self._balance = balance  # Protected Attribute
    self.account_no = account_no
    self.__PIN = PIN         # Private Attribute
    self.transactions = []   # List of tuples

    #update total_account as a new accout instance is created
    Account.total_accounts += 1     #OR self.__class__.total_accounts += 1

    # register this account in the registry. -> key = account_no & the value = actual Account object
    self.__class__.accounts_registry[account_no] = self 

  @property
  def balance(self):
      return self._balance

  @balance.setter
  def balance(self, new_balance):
      if new_balance < 0:
          print("Invalid: Balance can not be negative")
          return
      self._balance = new_balance

  def log_transaction(self, transaction_type, amount):
      self.transactions.append((transaction_type, amount))
      
  def account_info(self):
    return f"Bank: {self.__class__.bank_name}, Type: {self.account_type}, Owner: {self.owner}, Account Number: {self.mask_account()}, Balance: {self.balance} ETB"

  def get_available_funds(self):
      return self.balance

  # def check_balance(self):
  #   print(f"Your account {self.mask_account()} balance is {self.balance}")

  def mask_account(self):
    acc_no = str(self.account_no)
    mask = "*" * 4   # multiple a string by a number to repeat it (creats "****"")
    sliced = acc_no[4:] # slice from index 0 to 4
    return mask + sliced # concatinate the strings

  def authenticate(self, PIN):
    return self.__PIN == PIN

  def deposit(self, amount):
    if amount <= 0:
        print("Invalid Amount")
        return False
    self.balance += amount
    self.log_transaction('Deposit', amount)
    print(f"Your account {self.mask_account()} has been credited {amount} ETB. Your current balance is {self.balance}")
    return True
   
  def withdraw(self, amount, pin):
    if not self.authenticate(pin):
      print("Wrong PIN")
      return False

    if not isinstance(amount, (int, float)) or amount <= 0 :
      print("Invalid Amount")
      return False

    if self.get_available_funds() < amount:
       print('Insufficient Balance')
       return False
        
    self.balance -= amount
    self.log_transaction('Withdrawal', amount)
    print(f"Your account {self.mask_account()} has been debited {amount} ETB. Your current balance is {self.balance}")
    return True

  def transfer(self, target_account, amount, pin):
      if not self.authenticate(pin):
          print("Wrong PIN")
          return False
      if self.get_available_funds() < amount:
          print('Insufficient Balance')
          return False
      self.balance -= amount
      target_account.balance += amount
      self.log_transaction('Transfer', amount)
      print(f"Transferred {amount} from {self.owner}({self.mask_account()}) to {target_account.owner} ({target_account.mask_account()})")
      return True
    
  def transfer_by_accNo(self, target_account_no, amount, pin):
      if not self.authenticate(pin):
          print("Wrong PIN")
          return False
      target_account = self.__class__.find_account(target_account_no)
      if target_account is None:
          print("Target account not found")
          return False

      if self.get_available_funds() < amount:
          print("Insufficient funds")
          return False

      self.balance -= amount
      target_account.balance += amount
      self.log_transaction('Transfer', amount)
      print(f"Transferred {amount} from {self.owner}({self.mask_account()}) to {target_account.owner} ({target_account.mask_account()})")  
      return True

  def transaction_history(self):
    if not self.transactions:
      print('Transaction history is empty')
      return 
    print(f'--- Transaction History for {self.mask_account()} ---')
    #print(self.transaction)
    for transaction_type, amounts in self.transactions:
        print(f"{transaction_type} : {amounts} ETB")
    return True

  @abstractmethod
  def calculate_interest(Self):
      #calculates interest based on account type
      pass

  @property
  @abstractmethod
  def account_type(self):
      # returns specific type of account
      pass


In [2]:
class SavingAccount(Account):
    def __init__(self, owner, balance, account_no, PIN, interest_rate=0.07):
        super().__init__(owner,balance, account_no, PIN)
        self.interest_rate = interest_rate

    def calculate_interest(self):
        interest = self.balance * self.interest_rate
        print(f"Interest: {interest}.")
        self.deposit(interest)
    
    @property
    def account_type(self):
        return "Savings Account"
        

In [3]:
class CheckingAccount(Account):
    def __init__(self, owner, balance, account_no, PIN, overdraft_limit=500.00):
        super().__init__(owner, balance, account_no, PIN)
        self.__overdraft_limit = overdraft_limit

    @Account.balance.setter
    def balance(self, new_balance):
        if new_balance < -self.__overdraft_limit:
            print(f"Withdrawal exceeds overdraft limit of {self.get_overdraft_limit()}")
        self._balance = new_balance
        
    def set_overdraft_limit(self, limit):
        self.__overdraft_limit = limit

    def get_overdraft_limit(self):
        return self.__overdraft_limit

    def get_available_funds(self):
        return self.balance + self.get_overdraft_limit()

    def calculate_interest(self):
        # a checking account does not earn interest
        pass
         
    @property
    def account_type(self):
        return "Checking Account"
        

In [4]:
class JointAccount(Account):
    def __init__(self,owners, balance, account_no, PIN):
        super().__init__(", ".join(owners), balance, account_no, PIN)
        self.owners = owners

    # def account_info(self):
    #     return f"{super().account_info()} (Joint Owners: {})"
    def show_owners(self):
        print(f"Joint Account owners: {self.owners}")
    
    #checks is user is a valid owner
    def is_owner(self, owner_name):
        return owner_name in self.owners
        
    def calculate_interest(self):
        pass
         
    @property
    def account_type(self):
        return "Joint Account"
        

In [5]:
if __name__ == "__main__":
    # Creating Accounts
    acc1 = SavingAccount("Abebe", 1000, "1000123456", 1234)
    acc2 = CheckingAccount("Kebede", 2500, "1000789101", 0000)
    acc3 = JointAccount(["Samuel", "Selam"], 500, "1000234567", 9999)

    #Print All registered Accounts
    print(f"--- Registered Accounts ({Account.get_total_accounts()}) ---")
    Account.get_accounts()

    # Performing Transactions on account 1
    acc1.deposit(10000)
    acc1.withdraw(2000, 1234)
    acc2.calculate_interest()

    # Performing Transactions on account 2
    acc2.deposit(500)
    acc2.withdraw(3500, 0000)
    acc2.withdraw(200, 0000)
   
    # Performing Transaction on account 3
    acc3.deposit(3000)
    acc3.show_owners()

    # Performing transfers
    acc1.transfer(acc2, 500, 1234)
    acc2.transfer_by_accNo("1000234567", 200, 0000)

    # displaying Transaction History
    acc1.transaction_history()
    acc2.transaction_history()
    acc3.transaction_history()  

--- Registered Accounts (3) ---
Bank: CBE, Type: Savings Account, Owner: Abebe, Account Number: ****123456, Balance: 1000 ETB
Bank: CBE, Type: Checking Account, Owner: Kebede, Account Number: ****789101, Balance: 2500 ETB
Bank: CBE, Type: Joint Account, Owner: Samuel, Selam, Account Number: ****234567, Balance: 500 ETB
Your account ****123456 has been credited 10000 ETB. Your current balance is 11000
Your account ****123456 has been debited 2000 ETB. Your current balance is 9000
Your account ****789101 has been credited 500 ETB. Your current balance is 3000
Your account ****789101 has been debited 3500 ETB. Your current balance is -500
Insufficient Balance
Your account ****234567 has been credited 3000 ETB. Your current balance is 3500
Joint Account owners: ['Samuel', 'Selam']
Transferred 500 from Abebe(****123456) to Kebede (****789101)
Transferred 200 from Kebede(****789101) to Samuel, Selam (****234567)
--- Transaction History for ****123456 ---
Deposit : 10000 ETB
Withdrawal : 2000

**Next Steps**
- Refactor code into separate Python modules (move classes out of the notebook).  
- Add more account-related classes (e.g., User, Transaction, Budget).  
- Explore design patterns such as Factory (for creating accounts) and Strategy (for interest/fees).  
- Add persistence (saving account data to JSON or SQLite).  
- Expand towards a personal finance manager: budgets, expense tracking, and simple reports.
